Convert all stock data into USD

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path
import pandas as pd
CWD = Path.cwd().resolve()
PROJECT_ROOT = CWD if (CWD / "data").exists() else CWD.parent
DATA = PROJECT_ROOT / "data"

In [3]:
exchange = pd.read_csv(DATA / "raw"/ 'exchange_rate.csv')
exchange = exchange.dropna()
exchange["datadate"] = pd.to_datetime(exchange["datadate"])
exchange["year"] = exchange["datadate"].dt.year  
years = exchange["year"].unique()

In [4]:
exchange.rename(columns={"curd": "curcdd"}, inplace=True)
exchange.rename(columns={'exratd_tousd': 'exchange_rate'}, inplace=True)

In [5]:
results = []
for year in years:
    chunk = pd.read_csv(DATA / "raw"/ "stock_price" / "global_stock" / f"{year}.csv")
    chunk['datadate'] = pd.to_datetime(chunk['datadate'])
    chunk['year'] = chunk['datadate'].dt.year
    chunk['prccd_sh'] = chunk['prccd']/chunk['qunit']
    
    # Merge chunk with exchange rates
    chunk = chunk.merge(
        exchange[['datadate', 'curcdd', 'exchange_rate']],
        how='left',  
        on=['datadate', 'curcdd']  
)


    chunk['prccm'] =chunk['prccd_sh']*chunk['exchange_rate']
    
    temp_result = chunk.groupby(['gvkey', 'year']).agg({  
    'prccd_sh': 'mean',
    'prccm': 'mean',  
    'cshoc': 'mean',
        'qunit':'mean',
        'exchange_rate':'max'
}).reset_index()
   

    results.append(temp_result)  
   

In [6]:
final_result = pd.concat(results).groupby(['gvkey', 'year']).agg({  
    'prccm': 'mean',  
    'cshoc': 'mean',  
}).reset_index()  

In [7]:
final_result['cshoc'] = final_result["cshoc"].mul(1/1000000, axis=0)
final_result=final_result[final_result['year']<=2024]

In [8]:
final_result.to_csv(DATA / "global_stock_annual.csv", index=False)

In [9]:
year_inter = [1997,2000,2006,2010,2015,2020] #data are downloaded in intervals, 1997.csv contains data between 1997 and 1999,....

In [10]:
results_na = []
for year in year_inter:
    chunk = pd.read_csv(DATA / "raw"/ "stock_price" / "north_american_stock" / f"{year}.csv")
    chunk['datadate'] = pd.to_datetime(chunk['datadate'])
    chunk['year'] = chunk['datadate'].dt.year
    
    
    
    chunk = chunk.merge(
        exchange[['datadate', 'curcdd', 'exchange_rate']], 
        how='left', 
        on=['datadate', 'curcdd']  
)


    chunk['prccm'] =chunk['prccd']*chunk['exchange_rate']
    
   
    temp_result = chunk.groupby(['gvkey', 'year']).agg({  
    'prccd': 'mean',
    'prccm': 'mean',  
    'cshoc': 'mean',
        'exchange_rate':'max'
}).reset_index()
   

    results_na.append(temp_result)  

In [11]:
final_result_na = pd.concat(results_na).groupby(['gvkey', 'year']).agg({  
    'prccm': 'mean',  
    'cshoc': 'mean',  
}).reset_index()  

In [12]:
final_result_na['cshoc'] = final_result_na["cshoc"].mul(1/1000000, axis=0)

In [13]:
final_result_na.to_csv(DATA / "na_stock_annual.csv", index=False)